In [ ]:
!wget https://raw.githubusercontent.com/kdysunleo/NMT/main/eng-kor.txt

--2023-01-11 12:17:04--  https://raw.githubusercontent.com/kdysunleo/NMT/main/eng-kor.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 534332 (522K) [text/plain]
Saving to: ‘eng-kor.txt.1’

eng-kor.txt.1       100%[===================>] 521.81K  --.-KB/s    in 0.04s   

2023-01-11 12:17:04 (14.2 MB/s) - ‘eng-kor.txt.1’ saved [534332/534332]



**Sequence to Sequence 네트워크와 Attention을 이용한 Machine Translation 모델 구현**

- 한국어를 영어로 번역하도록 모델 학습

In [ ]:
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 21
random.seed(seed)
torch.manual_seed(seed)

언어별 정보 저장을 위한 클래스 생성

- 단어->아이디, 아이디->단어 사전 생성 (word2idx)
- addSentence 함수는 문장을 띄어쓰기를 기준으로 토큰화
- addWord 함수
    - word2idx, idx2word 만들기
    - 단어별 출현 횟수 구하기
    - 전체 단어의 개수 구하기


In [ ]:
SOS_token = 0
EOS_token = 1
UNK_token = 2

class Lang:
    def __init__(self, name):
        self.name = name
        ## 단어->아이디, 단어별 개수 사전 선언

        self.word2index = {}
        self.word2count = {}

        self.index2word = {0: "SOS", 1: "EOS", 2:"UNK"}
        self.n_words = 3  # Count SOS and EOS and UNK

    def addSentence(self, sentence):
        # sentence를 띄어쓰기 단위로 토큰화 하고, 토큰은 addWord 함수를 이용해서 정보 저장

        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        # word2idx, word2count, idx2word, nwords 정보 저장

        ####################  빈칸  ####################
        
        # 단어를 Dictionary에 추가 / 업데이트
        if word not in self.word2index: 
            self.word2index[word] = self.n_words # word에 idx를 mapping
            self.word2count[word] = 1 # 단어의 등장 횟수 = 1
            self.index2word[self.n_words] = word # idx에 word를 mapping
            self.n_words += 1 # 
        else:
            self.word2count[word] += 1 # 단어의 등장 횟수 + 1
            
        ###############################################

- 병렬 코퍼스 준비 : 영어-한국어
- 영어는 normalize를 진행
- readLangs 함수
    - 파일을 읽고 쌍으로 분리
    - Lang class를 이용하여 각 언어 정보 저장

In [ ]:
def normalizeString(s):
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def readLangs(lang1, lang2):
    print("Reading lines...")

    # git clone을 통해 데이터를 다운로드 받는 경우, 파일 경로 수정 필요
    f = open('%s-%s.txt' % (lang1, lang2), encoding='utf-8')
    lines = f.readlines()
    
    pairs = []
    for l in lines:
        line = l.split("\t")
        # 한국어, 영어
        pair = [line[1], normalizeString(line[0])]
        pairs.append(pair)

    input_lang = Lang(lang2)
    output_lang = Lang(lang1)
    f.close()
    return input_lang, output_lang, pairs

- MAX_LENGTH : 학습에 사용할 최대문장길이
- eng_prefixes : 해당 단어들로 시작하는 문장만 번역에 포함

- filterPair : 위 조건에 만족하는 pair만 반환
- filterPairs : 필터링된 pair만 저장해서 반환

In [ ]:
MAX_LENGTH = 30
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    ####################  빈칸  ####################

    # 영어는 해당하는 구문으로 시작하고 영어와 한국어 둘다 max_length보다 작은 데이터일 경우에만 True 반환
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH and p[1].startswith(eng_prefixes)
    
    ###############################################

def filterPairs(pairs):
    # filterPair가 True인 데이터만 저장후 반환
    return [pair for pair in pairs if filterPair(pair)]

데이터 준비 및 결과 확인
- **학습과 검증에 필요한 데이터쌍을 구분**
- 이전 단계에서 만든 클래스와 함수를 통해 결과 확인
- 전체 문장 개수(pair 개수)
- 필터링 후 문장개수(pair 개수)
- 각 언어별로, 사전에 등록된 단어 개수

In [ ]:
def prepareData(lang1, lang2):

    # 이전에 만든 클래스를 활용해서 영어 한국어의 정보를 저장
    # 조건에 맞게 데이터쌍을 필터링
    # 필터링된 데이터쌍의 80%는 학습용으로, 나머지 20%는 검증용으로 구분
    # 전체 데이터쌍의 수, 필터링 후 데이터쌍의 수 그리고 각 언어별 단어(토큰)의 개수를 출력
    
    input_lang, output_lang, pairs = readLangs(lang1, lang2)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))


    ####################  빈칸  ####################
    
    # 데이터 shuffle
    random.shuffle(pairs)

    # 데이터를 8:2로 split
    train_size = int(len(pairs)*0.8) 
    train_pairs = pairs[:train_size]
    valid_pairs = pairs[train_size:]

    # 각 언어 객체에 문장 추가
    for pair in train_pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])

    print("입력 언어 : ",input_lang.name, input_lang.n_words)
    print("출력 언어 : ",output_lang.name, output_lang.n_words)
    
    ###############################################

    return input_lang, output_lang, train_pairs, valid_pairs


input_lang, output_lang, train_pairs, valid_pairs = prepareData('eng', 'kor')
print(random.choice(train_pairs))

Reading lines...
Read 3648 sentence pairs
Trimmed to 223 sentence pairs
입력 언어 :  kor 439
출력 언어 :  eng 308
['나는 내년에 호주에 간다.', 'i m going to australia next year .']


인코더 역할의 RNN 생성
- torch.nn.Embedding으로 랜덤 임베딩 생성
- 인풋의 임베딩을 만들고 배치차원 늘려주기
- 임베딩을 gru에 넣고 output과 hidden state 얻기

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        embedding_size = hidden_size
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.gru = nn.GRU(embedding_size, hidden_size)

    def forward(self, input, hidden):
        ####################  빈칸  ####################

        # 임베딩 만들고, gru에 입력으로 넣어주고 output 과 hidden 얻기
        embedded = self.embedding(input).view(1, 1, -1) # word idx으로부터 word embedding 만들기
        output, hidden = self.gru(embedded, hidden) # gru

        ###############################################

        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

디코더 역할의 RNN 생성

- torch.nn.Embedding으로 랜덤 임베딩 생성
- 인풋의 임베딩을 만들고 배치차원 늘려주기
- 임베딩을 gru에 넣고 output과 hidden state 얻기
- softmax를 이용하여 전체 단어에서 가장 확률높은 단어 뽑기


In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()

        ####################  빈칸  ####################
        # GRU의 결과가 linear layer 와 softmax를 거쳐야 한다
        # 차원 정의
        self.hidden_size = hidden_size
        embedding_size = hidden_size
        # layer 정의
        self.embedding = nn.Embedding(output_size, embedding_size)
        self.gru = nn.GRU(embedding_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        # log prob 계산
        self.log_softmax = nn.LogSoftmax(dim=1)
        
        ###############################################

    def forward(self, input, hidden):

        ####################  빈칸  ####################
        # 임베딩 만들고, gru 입력으로 넣어주고, 
        # output을 linear 와 softmax의 입력으로 넣어주기

        # batch_size = 1
        # input (단어의 index) : batch_size X 1
        # self.embedding(input) : batch_size X embedding_size    (embedding_size = hidden_size)
        # output = self.embedding(input).view(1, 1, -1) : batch_size X seq_len x embedding_size (batch_size = 1)
        # hidden (이전 step의 hidden vector) : batch_size X hidden_size
        # output, hidden = self.gru(output, hidden) : 각각 batch_size X hidden_size
        # word embedding (1 X 1 X Embedding_size)

        output = self.embedding(input).view(1, 1, -1) # word idx으로부터 word embedding 만들기
        output, hidden = self.gru(output, hidden) # gru, encoder의 hidden을 init

        # attention 연산이 들어갈 부분
        
        output = self.log_softmax(self.out(output[0])) # decoder gru의 output으로 log prob을 계산

        ###############################################

        return output, hidden

#    def initHidden(self):
#        return torch.zeros(1, 1, self.hidden_size, device=device)
        

학습 데이터 준비
- 각 문장마다 word2idx를 이용하여 tensor로 변환(벡터화)



In [ ]:
def indexesFromSentence(lang, sentence):
    ####################  빈칸  ####################

    # 문장을 띄어쓰기 단위로 토큰화 하고, 해당 단어(토큰)의 id로 변환
    # 단, 검증때 사전에 없는 단어가 출현할 수 있는 상황도 처리할 수 있어야 함
    result = []
    for word in sentence.split(' '):
        if word in lang.word2index:
            result.append(lang.word2index[word]) # 각 단어에 해당하는 idx 추가
        else:
            result.append(UNK_token) # 해당 단어가 존재하지 않는 경우 UNK token 추가
    return result

    ###############################################


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)

    ####################  빈칸  ####################

    # 실제 데이터 외에 추가로 들어가야할 토큰이 있다!
    indexes.append(EOS_token) # EOS token 추가

    ###############################################

    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

모델 학습 코드


In [ ]:
teacher_forcing_ratio = 0.5

# train for each step
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion, max_length=MAX_LENGTH):
    # encoder 초기값 설정, gradient 0 으로 초기화
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    ####################  빈칸  ####################
    # encoder 호출
    for ei in range(input_length):
        # 각 token에 대해서 hidden과 output을 계산
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        
    # decoder의 초기 input과 hidden값 결정
    decoder_input = torch.tensor([[SOS_token]], device=device) # SOS 토큰
    decoder_hidden = encoder_hidden # encoder의 hidden을 decoder의 초기 hidden으로 사용
    ###############################################

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        ####################  빈칸  ####################
        # 디코더의 다음 입력으로 실제값 사용
        for di in range(target_length):
            # decoder로부터 hidden과 output을 계산
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di]) # loss 계산
            # 다음 decoder의 input은 정답으로부터 가져옴
            decoder_input = target_tensor[di]
        ###############################################

    else:
        ####################  빈칸  ####################

        # 디코더의 다음 입력으로 예측값 사용
        for di in range(target_length):
            # decoder로부터 hidden과 output을 계산
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            
            # decoder의 output으로부터 가장 확률이 높은 token을 다음 decoder input으로 사용
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.detach()
            
            loss += criterion(decoder_output, target_tensor[di]) # loss 계산
            # decoder가 EOS token을 생성하면 종료
            if decoder_input.item() == EOS_token:
                break
        ###############################################

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

시간측정 출력 함수

In [ ]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

학습 진행 과정
- 타이머 시작
- optimizer, criterion 선언
- 전체 학습 데이터에 iteration 수만큼 랜덤하게 데이터 구축

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, learning_rate=0.01):
    start = time.time()
    print_loss_total = 0  # Reset every print_every

    ####################  빈칸  ####################
    # optimizer 선언, 데이터 준비, loss함수 선언  
    # optimizer 선언
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate) # Adam / AdamW
    # 학습 데이터에서 random하게 선택
    training_pairs = [tensorsFromPair(random.choice(train_pairs))
                      for i in range(n_iters)]
    # Negative Log Likelihood Loss 정의
    criterion = nn.NLLLoss()
    ###############################################
    # n_iter 횟수만큼 모델 학습 및 로스 출력
    for iter in range(1, n_iters + 1):
      
        ####################  빈칸  ####################
        # 학습에 사용할 데이터 input, target 가져오기
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        # 학습 및 loss 반환
        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        
        ###############################################

        print_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))


In [ ]:
hidden_size = 256

####################  빈칸  ####################
# encoder, decoder를 선언하고, 7500번의 iteration으로 학습하고 1000번마다 loss 출력
# 모델 정의
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

# 학습
trainIters(encoder1, decoder1, 7500, print_every=1000)
###############################################

0m 11s (- 1m 12s) (1000 13%) 2.6594
0m 18s (- 0m 52s) (2000 26%) 1.9629
0m 27s (- 0m 40s) (3000 40%) 1.3645
0m 35s (- 0m 30s) (4000 53%) 0.7722
0m 43s (- 0m 21s) (5000 66%) 0.3931
0m 51s (- 0m 12s) (6000 80%) 0.2011
0m 59s (- 0m 4s) (7000 93%) 0.1191


모델 검증 코드

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():

        ####################  빈칸  ####################

        # input 데이터에서 문장 가져와서 텐서로 변환 (train과 동일)
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]

        # 인코더의 초기 hidden state를 결정 (train과 동일)
        encoder_hidden = encoder.initHidden()
        
        ###############################################

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)

        ####################  빈칸  ####################

        # decoder의 초기 input 과 hidden state 결정 (train과 동일)
        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden

        ###############################################
        
        decoded_words = []
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)

            ####################  빈칸  ####################

            # decoding 멈출 시기 결정, 예측단어 decoded_words에 저장

            # EOS_token을 생성하면 종료
            # topi = tensor([3])
            # topi.item() = 3
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                # 예측한 단어를 추가
                decoded_words.append(output_lang.index2word[topi.item()])

            ###############################################
            
            decoder_input = topi
        return decoded_words

검증 데이터쌍에서 결과 확인

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

smoothie = SmoothingFunction().method4 # 짧은 문장의 score가 너무 높아지는 것을 방지

# BLEU score (n-gram)
# 예측 : A B C D E
# 정답 : A B C E D
# 1-gram A / B / C / D / E
# 2-gram (A, B) / (B, C)
# 3-gram
# 4-gram

eval_pair = [['너 건망증이 좀 있구나.',"You're quite forgetful."],
             ['미안한데, 내가 도와줄 수가 없어.',"I'm sorry, I can't help you."],
             ['너 떨고 있네.',"You're shivering."],
             ['그는 우울하다.',"He is depressed."],
             ['난 네 선생이다.',"I'm your teacher."],
             ['내 피가 끓고 있었다.	',"My blood was boiling."]]

def evaluatePrint(encoder, decoder):
    for pair in eval_pair:
        print('원본: ', pair[0])
        print('정답 변역: ', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words[:-1])
        print('예측 번역: ', output_sentence)
        print("sentence_bleu: %.3f" % (sentence_bleu([normalizeString(pair[1]).split()], output_sentence.split(), smoothing_function=smoothie)))
        print('')

전체 검증 데이터의 BLEU score 평균 확인

In [ ]:
def evaluateScore(encoder, decoder):
    score = 0.0
    for pair in valid_pairs:
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words[:-1])
        score += sentence_bleu([normalizeString(pair[1]).split()], output_sentence.split(), smoothing_function=smoothie)
    print('Score : ',score/len(valid_pairs))

In [ ]:
evaluatePrint(encoder1, decoder1)
evaluateScore(encoder1, decoder1)

원본:  너 건망증이 좀 있구나.
정답 변역:  You're quite forgetful.
예측 번역:  you re quite forgetful .
sentence_bleu: 1.000

원본:  미안한데, 내가 도와줄 수가 없어.
정답 변역:  I'm sorry, I can't help you.
예측 번역:  i m sorry i can t help you .
sentence_bleu: 1.000

원본:  너 떨고 있네.
정답 변역:  You're shivering.
예측 번역:  i m sorry .
sentence_bleu: 0.061

원본:  그는 우울하다.
정답 변역:  He is depressed.
예측 번역:  she s depressed .
sentence_bleu: 0.168

원본:  난 네 선생이다.
정답 변역:  I'm your teacher.
예측 번역:  i m sorry .
sentence_bleu: 0.145

원본:  내 피가 끓고 있었다.	
정답 변역:  My blood was boiling.
예측 번역:  i m sorry but there s no other way .
sentence_bleu: 0.023

Score :  0.13787596384361508


Attention 기법을 적용한 디코더 RNN
- dot production 으로 attention score 구하기

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.softmax = nn.Softmax(dim=1)
        self.out = nn.Linear(self.hidden_size*2, self.output_size)
        self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, encoder_hiddens, input_length):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        ####################  빈칸  ####################
        # gru를 돌고 나온 hidden state로 어텐션 구하기
        # attn_score (dot) -> attn distribution -> attn value -> concat (attn value;hidden state)
        # 이때, 실제 input 길이를 제외한 나머지 부분은 매우 작은 값(-9e10)으로 마스킹해준 후 softmax를 돌아야함
        # concat 된 벡터를 linear 하나 돌아서 output 도출
        # output, hidden 계산
        output, hidden = self.gru(embedded, hidden)
        # output, hidden : batch_size(1) x 1 x hidden_size
        # attn score 구하기 (내적)
        attn_score = torch.mm(hidden.squeeze(0), encoder_hiddens)
        # hidden.squeeze(0) : 1 x hidden_size     ex) [[1, 2]] : 1 X 2 -> [1, 2] : 2
        # encoder_hiddens : hidden_size x input_length
        # attn_score : (1 x hidden_size) x (hidden_size x input_length) => 1 x input_length
        # input을 제외한 부분은 masking
        mask = torch.full((1,self.max_length-input_length), -9e10)
        attn_score[0][input_length:]=mask
        # softmax를 통해 encoder의 hidden embedding들을 weight sum
        attn_dist = self.softmax(attn_score)
        # 분포가 확률분포로 변함, 차원 변환 X
        # attn_dist : 1 x input_length
        attn_value = torch.mm(attn_dist, torch.transpose(encoder_hiddens,0,1))
        # torch.transpose(encoder_hiddens,0,1) : input_length X hidden_size     0, 1차원을 변환
        # attn_value = (1 x input_length) X (input_length x hidden_size) => 1 x hidden_size
        # hidden과 encoder hidden에 어텐션을 적용한 결과를 concat하여 최종 단어 예측
        output =  torch.cat((hidden.squeeze(0), attn_value), dim=1)
        # hidden.squeeze(0) : 1 x hidden_size
        # torch.cat(([1, 2], [3, 4])) => [1, 2, 3, 4]
        # torch.cat((hidden.squeeze(0), attn_value), dim=1) => 1 x 2 hidden_size
        output = self.log_softmax(self.out(output))
        ###############################################


        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

어텐션 모델 학습 코드

In [ ]:
teacher_forcing_ratio = 0.5

# train for each step
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):

    ####################  빈칸  ####################

    # 기본 decoder를 사용했을 때와 거의 유사하다.
    # decoder 인자에 주의하며, 위에서 작성한 코드를 참고해서 작성
    # encoder 의 hiddens state 값들을 모아두어야 함, 미리 transpose 시키기

    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_hiddens = torch.zeros(max_length, encoder.hidden_size, device=device)
    # 30 X 256 차원 0 텐서

    loss = 0

    # encoder 호출
    for ei in range(input_length):
    
        # 기존의 decoder 학습과 달라진 점: 모든 encdoer의 hidden을 저장 (decoder의 attn에 사용)
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_hiddens[ei] = encoder_hidden.view(-1, encoder.hidden_size) # 1 X 256 차원

    encoder_hiddens = torch.transpose(encoder_hiddens, 0, 1) # 256 x 30 차원

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # decoder의 input에 encoder의 모든 hidden embedding을 사용
    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_hiddens, input_length)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_hiddens, input_length)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.detach()
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()
    
    ###############################################

    return loss.item() / target_length

어텐션 모델 검증 코드

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    ####################  빈칸  ####################

    # 기본 decoder를 사용했을 때와 거의 유사하다.
    # decoder 인자에 주의하며, 위에서 작성한 코드를 참고해서 작성
    # encoder 의 hiddens state 값들을 모아두어야 함, 미리 transpose 시키기

    # 기존의 decoder 학습과 달라진 점: 모든 encdoer의 hidden을 저장 (decoder의 attn에 사용)
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_hiddens = torch.zeros(max_length, encoder.hidden_size, device=device)

        # encoder 호출
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(
                input_tensor[ei], encoder_hidden)
            encoder_hiddens[ei] = encoder_hidden.view(-1, encoder.hidden_size)
        encoder_hiddens = torch.transpose(encoder_hiddens, 0, 1)

        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_hiddens, input_length)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            decoder_input = topi

      ###############################################
      
        return decoded_words

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 7500, print_every=1000) 

0m 12s (- 1m 19s) (1000 13%) 2.2779
0m 25s (- 1m 9s) (2000 26%) 0.9982
0m 38s (- 0m 57s) (3000 40%) 0.3078
0m 50s (- 0m 44s) (4000 53%) 0.1065
1m 3s (- 0m 31s) (5000 66%) 0.0648
1m 16s (- 0m 19s) (6000 80%) 0.0370
1m 29s (- 0m 6s) (7000 93%) 0.0367


In [ ]:
evaluatePrint(encoder1, attn_decoder1)
evaluateScore(encoder1, attn_decoder1)

원본:  너 건망증이 좀 있구나.
정답 변역:  You're quite forgetful.
예측 번역:  you re quite forgetful .
sentence_bleu: 1.000

원본:  미안한데, 내가 도와줄 수가 없어.
정답 변역:  I'm sorry, I can't help you.
예측 번역:  i m sorry i can t help you .
sentence_bleu: 1.000

원본:  너 떨고 있네.
정답 변역:  You're shivering.
예측 번역:  i m sorry but i don t you ?
sentence_bleu: 0.026

원본:  그는 우울하다.
정답 변역:  He is depressed.
예측 번역:  she s depressed .
sentence_bleu: 0.168

원본:  난 네 선생이다.
정답 변역:  I'm your teacher.
예측 번역:  i m a sorry i kept you .
sentence_bleu: 0.079

원본:  내 피가 끓고 있었다.	
정답 변역:  My blood was boiling.
예측 번역:  i m sorry but i don t you ?
sentence_bleu: 0.000

Score :  0.19931846065660075
